In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.tree import DecisionTreeClassifier


from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


from sklearn.tree import export_graphviz
from graphviz import Source

from sklearn.pipeline import Pipeline, make_pipeline

import pandas_profiling

Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 255 ('font.family:AppleGothic')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 256 ('font.size:12')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 370 ('axes.titlesize:     large   # fontsize of the axes title')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 376 ('axes.labelsize:     medium  # fontsize of the x any y labels')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 410 ('axes.unicode_minus: True  # use Unicode for the minus symbol rather than hyphen.  See')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib

In [2]:
combats_df = pd.read_csv('./combats.csv')
pokemon_df = pd.read_csv('./pokemon.csv')
test_df = pd.read_csv('./tests.csv')

In [3]:
pokemon_df.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk',
       'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [4]:
pokemon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        799 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 69.7+ KB


In [5]:
pokemon_df.describe()

,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
count,800.0000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000
mean,400.5000,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375
std,231.0844,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129
min,1.0000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000
25%,200.7500,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000
50%,400.5000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000
75%,600.2500,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000
max,800.0000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000


In [6]:
cat_cols = ['Type 1', 'Type 2', 'Generation', 'Legendary']
num_cols = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',]
identical_cols = ['#', 'Name']

#### null

##### Type 1

In [7]:
pokemon_df.isnull().sum()

#               0
Name            1
Type 1          0
Type 2        386
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [8]:
pokemon_df.loc[pokemon_df['Name'].isnull(), 'Name'] = 'Primeape'
pokemon_df.iloc[62]

#                   63
Name          Primeape
Type 1        Fighting
Type 2             NaN
HP                  65
Attack             105
Defense             60
Sp. Atk             60
Sp. Def             70
Speed               95
Generation           1
Legendary        False
Name: 62, dtype: object

In [9]:
pokemon_df.isnull().sum()

#               0
Name            0
Type 1          0
Type 2        386
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

##### Type 2

In [10]:
pokemon_df.loc[pokemon_df['Type 2'].isnull()]

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
5,6,Charmeleon,Fire,NaN,58,64,58,80,65,80,1,False
9,10,Squirtle,Water,NaN,44,48,65,50,64,43,1,False
10,11,Wartortle,Water,NaN,59,63,80,65,80,58,1,False
11,12,Blastoise,Water,NaN,79,83,100,85,105,78,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
775,776,Sliggoo,Dragon,NaN,68,75,53,83,113,60,6,False
776,777,Goodra,Dragon,NaN,90,100,70,110,150,80,6,False
788,789,Bergmite,Ice,NaN,55,69,85,32,35,28,6,False
789,790,Avalugg,Ice,NaN,95,117,184,44,46,28,6,False


In [11]:
pokemon_df['Type 2'].value_counts(dropna=False)

NaN         386
Flying       97
Ground       35
Poison       34
Psychic      33
Fighting     26
Grass        25
Fairy        23
Steel        22
Dark         20
Dragon       18
Ghost        14
Ice          14
Water        14
Rock         14
Fire         12
Electric      6
Normal        4
Bug           3
Name: Type 2, dtype: int64

In [20]:
# 우선 기준을 찾기힘드므로, 모두 None 값 넣어주기
pokemon_df.fillna(value='None', inplace=True)

In [ ]:
# pokemon_df.dropna(axis=0, inplace=True)

In [21]:
pokemon_df['Type 2']

0      Poison
1      Poison
2      Poison
3      Poison
4        None
        ...  
795     Fairy
796     Fairy
797     Ghost
798      Dark
799     Water
Name: Type 2, Length: 800, dtype: object

In [22]:
pokemon_df.loc[pokemon_df['Type 2'].isnull()]

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary


In [23]:
pokemon_df[pokemon_df['Type 1']=='Normal']['Type 2'].value_counts(dropna=False)

None        61
Flying      24
Fairy        5
Psychic      2
Grass        2
Fighting     2
Ground       1
Water        1
Name: Type 2, dtype: int64

In [24]:
pokemon_df.loc[pokemon_df['Type 2'].isnull()]['Type 1'].value_counts(dropna=False)

Series([], Name: Type 1, dtype: int64)

In [25]:
pokemon_df['Type 2'].isnull().sum()

0

#### Legendary

In [26]:
pokemon_df['Legendary']

0      False
1      False
2      False
3      False
4      False
       ...  
795     True
796     True
797     True
798     True
799     True
Name: Legendary, Length: 800, dtype: bool

In [27]:
pokemon_df['Legendary']=pokemon_df['Legendary'].apply(lambda x: 1 if x == True else 0)
pokemon_df

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,0
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,0
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,0
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,0
4,5,Charmander,Fire,None,39,52,43,60,50,65,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
795,796,Diancie,Rock,Fairy,50,100,150,100,150,50,6,1
796,797,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,1
797,798,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,1
798,799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,1


### One Hot Encoding

In [ ]:
# pokemon_df.columns

In [ ]:
# encoding_cols = ['Type 1', 'Type 2']
# not_encoding_cols = ['#', 'Name', 'HP', 'Attack', 'Defense', 'Sp. Atk',
#        'Sp. Def', 'Speed', 'Generation', 'Legendary']

In [ ]:
# poke_dummies = pd.get_dummies(pokemon_df, columns=encoding_cols)
# poke_dummies

In [ ]:
# poke_dummies

### Label Encoding

In [31]:
encoding_cols = ['Type 1', 'Type 2']
not_encoding_cols = ['#', 'Name', 'HP', 'Attack', 'Defense', 'Sp. Atk',
       'Sp. Def', 'Speed', 'Generation', 'Legendary']

In [32]:
# 각 칼럼(Feature)마다 변환 전의 고유값들을 저장한 딕셔너리
enc_dict = {}

def encoding_label(x):
    # 범주형 Series를 매개변수로 받아서 LabelEncoding 처리후 반환
    le = LabelEncoder()
    r = le.fit_transform(x) # 1차원
    
    enc_dict[x.name] = le.classes_ # x.name: 컬럼명
    return r

In [33]:
pokemon_df_enc = pokemon_df[encoding_cols].apply(encoding_label)
pokemon_df_enc

,Type 1,Type 2
0,9,14
1,9,14
2,9,14
3,9,14
4,6,12
...,...,...
795,15,4
796,15,4
797,14,8
798,14,1


In [34]:
pokemon_df_enc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Type 1  800 non-null    int64
 1   Type 2  800 non-null    int64
dtypes: int64(2)
memory usage: 12.6 KB


In [35]:
enc_dict['Type 1'][0]

'Bug'

In [36]:
pokemon_df_le = pd.concat([pokemon_df_enc, pokemon_df[not_encoding_cols]], axis=1)
pokemon_df_le

,Type 1,Type 2,#,Name,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,9,14,1,Bulbasaur,45,49,49,65,65,45,1,0
1,9,14,2,Ivysaur,60,62,63,80,80,60,1,0
2,9,14,3,Venusaur,80,82,83,100,100,80,1,0
3,9,14,4,Mega Venusaur,80,100,123,122,120,80,1,0
4,6,12,5,Charmander,39,52,43,60,50,65,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
795,15,4,796,Diancie,50,100,150,100,150,50,6,1
796,15,4,797,Mega Diancie,50,160,110,160,110,110,6,1
797,14,8,798,Hoopa Confined,80,110,60,150,130,70,6,1
798,14,1,799,Hoopa Unbound,80,160,60,170,130,80,6,1


In [37]:
pokemon_df_le.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Type 1      800 non-null    int64 
 1   Type 2      800 non-null    int64 
 2   #           800 non-null    int64 
 3   Name        800 non-null    object
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    int64 
dtypes: int64(11), object(1)
memory usage: 75.1+ KB


In [40]:
# name drop
name_df=pokemon_df_le['Name']
name_df.to_frame()

,Name
0,Bulbasaur
1,Ivysaur
2,Venusaur
3,Mega Venusaur
4,Charmander
...,...
795,Diancie
796,Mega Diancie
797,Hoopa Confined
798,Hoopa Unbound


In [41]:
pokemon_df_le.drop('Name', axis=1, inplace=True)
pokemon_df_le

,Type 1,Type 2,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,9,14,1,45,49,49,65,65,45,1,0
1,9,14,2,60,62,63,80,80,60,1,0
2,9,14,3,80,82,83,100,100,80,1,0
3,9,14,4,80,100,123,122,120,80,1,0
4,6,12,5,39,52,43,60,50,65,1,0
...,...,...,...,...,...,...,...,...,...,...,...
795,15,4,796,50,100,150,100,150,50,6,1
796,15,4,797,50,160,110,160,110,110,6,1
797,14,8,798,80,110,60,150,130,70,6,1
798,14,1,799,80,160,60,170,130,80,6,1


### int -> string
- '#','Generation','Legendary' 세 칼럼은 숫자가 크기를 나타내는 것이 아니므로 string 형태로 변환해준다.
- 거리기반(KNN, SVM, 등 사용시 해줄 것)
    - 현재는 트리 기반의 모델을 사용해보자

In [ ]:
# poke_dummies[['#','Generation','Legendary','Type 1', 'Type 2']] = poke_dummies[['#','Generation','Legendary', 'Type 1', 'Type 2']].astype(str)

In [ ]:
# pokemon_df_le[['#','Generation','Legendary','Type 1', 'Type 2']].info()

In [ ]:
# pokemon_df_le.info()

In [ ]:
# pokemon_df_le

### combats_df

#### Winner_mark

In [42]:
combats_df['Winner_mark'] = combats_df.apply(lambda x:1 if x['First_pokemon'] == x['Winner'] else 2, axis=1)
combats_df

,First_pokemon,Second_pokemon,Winner,Winner_mark
0,266,298,298,2
1,702,701,701,2
2,191,668,668,2
3,237,683,683,2
4,151,231,151,1
...,...,...,...,...
49995,707,126,707,1
49996,589,664,589,1
49997,303,368,368,2
49998,109,89,109,1


In [52]:
pokemon_df_le.head()

,Type 1,Type 2,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,9,14,1,45,49,49,65,65,45,1,0
1,9,14,2,60,62,63,80,80,60,1,0
2,9,14,3,80,82,83,100,100,80,1,0
3,9,14,4,80,100,123,122,120,80,1,0
4,6,12,5,39,52,43,60,50,65,1,0


### 전처리 완료된 데이터 프레임 생성

##### first

In [53]:
temp_first_df = pd.merge(combats_df, pokemon_df_le,
                         how = 'left', left_on='First_pokemon', right_on='#'
                        )
temp_first_df.columns

Index(['First_pokemon', 'Second_pokemon', 'Winner', 'Winner_mark', 'Type 1',
       'Type 2', '#', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',
       'Generation', 'Legendary'],
      dtype='object')

In [54]:
temp_first_df.sort_values(by='#')

,First_pokemon,Second_pokemon,Winner,Winner_mark,Type 1,Type 2,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
15176,1,372,372,2,9,14,1,45,49,49,65,65,45,1,0
45814,1,462,462,2,9,14,1,45,49,49,65,65,45,1,0
17362,1,6,6,2,9,14,1,45,49,49,65,65,45,1,0
15251,1,649,649,2,9,14,1,45,49,49,65,65,45,1,0
4883,1,285,1,1,9,14,1,45,49,49,65,65,45,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18744,800,1,800,1,6,18,800,80,110,120,130,90,70,6,1
33517,800,280,280,2,6,18,800,80,110,120,130,90,70,6,1
8183,800,132,132,2,6,18,800,80,110,120,130,90,70,6,1
5272,800,157,157,2,6,18,800,80,110,120,130,90,70,6,1


In [55]:
temp_first_df=temp_first_df[['#', 'First_pokemon', 'Second_pokemon', 'Winner', 'Winner_mark', 'Type 1',
       'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',
       'Generation', 'Legendary']]
temp_first_df

,#,First_pokemon,Second_pokemon,Winner,Winner_mark,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,266,266,298,298,2,15,10,50,64,50,45,50,41,2,0
1,702,702,701,701,2,9,5,91,90,72,90,129,108,5,1
2,191,191,668,668,2,4,7,55,40,85,80,105,40,2,0
3,237,237,683,683,2,6,12,40,40,40,70,40,20,2,0
4,151,151,231,151,1,15,18,70,60,125,115,70,55,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,707,707,126,707,1,2,6,100,120,100,150,120,90,5,1
49996,589,589,664,589,1,10,12,60,85,40,30,45,68,5,0
49997,303,303,368,368,2,17,7,60,50,100,85,70,65,3,0
49998,109,109,89,109,1,3,12,40,30,50,55,55,100,1,0


In [56]:
temp_first_df.columns= ['#','First_pokemon', 'Second_pokemon', 'Winner', 'Winner_mark', 
                               'Type 1_first','Type 2_first', 'HP_first', 'Attack_first', 'Defense_first',
                               'Sp. Atk_first', 'Sp. Def_first', 'Speed_first',
                               'Generation_first', 'Legendary_first']
temp_first_df

,#,First_pokemon,Second_pokemon,Winner,Winner_mark,Type 1_first,Type 2_first,HP_first,Attack_first,Defense_first,Sp. Atk_first,Sp. Def_first,Speed_first,Generation_first,Legendary_first
0,266,266,298,298,2,15,10,50,64,50,45,50,41,2,0
1,702,702,701,701,2,9,5,91,90,72,90,129,108,5,1
2,191,191,668,668,2,4,7,55,40,85,80,105,40,2,0
3,237,237,683,683,2,6,12,40,40,40,70,40,20,2,0
4,151,151,231,151,1,15,18,70,60,125,115,70,55,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,707,707,126,707,1,2,6,100,120,100,150,120,90,5,1
49996,589,589,664,589,1,10,12,60,85,40,30,45,68,5,0
49997,303,303,368,368,2,17,7,60,50,100,85,70,65,3,0
49998,109,109,89,109,1,3,12,40,30,50,55,55,100,1,0


In [57]:
temp_first_df.columns

Index(['#', 'First_pokemon', 'Second_pokemon', 'Winner', 'Winner_mark',
       'Type 1_first', 'Type 2_first', 'HP_first', 'Attack_first',
       'Defense_first', 'Sp. Atk_first', 'Sp. Def_first', 'Speed_first',
       'Generation_first', 'Legendary_first'],
      dtype='object')

##### second

In [71]:
temp_df = pd.merge(temp_first_df, pokemon_df_le, \
         how = 'left', left_on='Second_pokemon', right_on='#' )
temp_df

,#_x,First_pokemon,Second_pokemon,Winner,Winner_mark,Type 1_first,Type 2_first,HP_first,Attack_first,Defense_first,...,Type 2,#_y,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,266,266,298,298,2,15,10,50,64,50,...,1,298,70,70,40,60,40,60,3,0
1,702,702,701,701,2,9,5,91,90,72,...,5,701,91,129,90,72,90,108,5,1
2,191,191,668,668,2,4,7,55,40,85,...,12,668,75,75,75,125,95,40,5,0
3,237,237,683,683,2,6,12,40,40,40,...,12,683,77,120,90,60,90,48,5,0
4,151,151,231,151,1,15,18,70,60,125,...,16,231,20,10,230,10,230,5,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,707,707,126,707,1,2,6,100,120,100,...,12,126,30,40,70,70,25,60,1,0
49996,589,589,664,589,1,10,12,60,85,40,...,12,664,35,55,40,45,40,60,5,0
49997,303,303,368,368,2,17,7,60,50,100,...,12,368,73,115,60,60,60,90,3,0
49998,109,109,89,109,1,3,12,40,30,50,...,17,89,25,35,70,95,55,45,1,0


In [72]:
temp_df.columns

Index(['#_x', 'First_pokemon', 'Second_pokemon', 'Winner', 'Winner_mark',
       'Type 1_first', 'Type 2_first', 'HP_first', 'Attack_first',
       'Defense_first', 'Sp. Atk_first', 'Sp. Def_first', 'Speed_first',
       'Generation_first', 'Legendary_first', 'Type 1', 'Type 2', '#_y', 'HP',
       'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation',
       'Legendary'],
      dtype='object')

In [73]:
temp_df[['#_x', 'First_pokemon', 'Second_pokemon', 'Winner', 'Winner_mark','#_y']]

,#_x,First_pokemon,Second_pokemon,Winner,Winner_mark,#_y
0,266,266,298,298,2,298
1,702,702,701,701,2,701
2,191,191,668,668,2,668
3,237,237,683,683,2,683
4,151,151,231,151,1,231
...,...,...,...,...,...,...
49995,707,707,126,707,1,126
49996,589,589,664,589,1,664
49997,303,303,368,368,2,368
49998,109,109,89,109,1,89


In [75]:
temp_df = temp_df[['#_x', 'First_pokemon', 'Second_pokemon', 'Winner', 'Winner_mark',
                   'Type 1_first', 'Type 2_first', 'HP_first', 'Attack_first',
                   'Defense_first', 'Sp. Atk_first', 'Sp. Def_first', 'Speed_first',
                   'Generation_first', 'Legendary_first','#_y', 'Type 1',
                   'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',
                   'Generation', 'Legendary']]
temp_df

,#_x,First_pokemon,Second_pokemon,Winner,Winner_mark,Type 1_first,Type 2_first,HP_first,Attack_first,Defense_first,...,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,266,266,298,298,2,15,10,50,64,50,...,9,1,70,70,40,60,40,60,3,0
1,702,702,701,701,2,9,5,91,90,72,...,15,5,91,129,90,72,90,108,5,1
2,191,191,668,668,2,4,7,55,40,85,...,14,12,75,75,75,125,95,40,5,0
3,237,237,683,683,2,6,12,40,40,40,...,2,12,77,120,90,60,90,48,5,0
4,151,151,231,151,1,15,18,70,60,125,...,0,16,20,10,230,10,230,5,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,707,707,126,707,1,2,6,100,120,100,...,17,12,30,40,70,70,25,60,1,0
49996,589,589,664,589,1,10,12,60,85,40,...,3,12,35,55,40,45,40,60,5,0
49997,303,303,368,368,2,17,7,60,50,100,...,12,12,73,115,60,60,60,90,3,0
49998,109,109,89,109,1,3,12,40,30,50,...,3,17,25,35,70,95,55,45,1,0


In [76]:
temp_df.columns

Index(['#_x', 'First_pokemon', 'Second_pokemon', 'Winner', 'Winner_mark',
       'Type 1_first', 'Type 2_first', 'HP_first', 'Attack_first',
       'Defense_first', 'Sp. Atk_first', 'Sp. Def_first', 'Speed_first',
       'Generation_first', 'Legendary_first', '#_y', 'Type 1', 'Type 2', 'HP',
       'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation',
       'Legendary'],
      dtype='object')

In [77]:
# rename cols
temp_df.columns = ['#_x', 'First_pokemon', 'Second_pokemon', 'Winner', 'Winner_mark',
       'Type 1_first', 'Type 2_first', 'HP_first', 'Attack_first',
       'Defense_first', 'Sp. Atk_first', 'Sp. Def_first', 'Speed_first',
       'Generation_first', 'Legendary_first', '#_y', 'Type 1_second',
       'Type 2_second', 'HP_second', 'Attack_second', 'Defense_second', 'Sp. Atk_second', 'Sp. Def_second', 'Speed_second',
       'Generation_second', 'Legendary_second']
temp_df.columns

Index(['#_x', 'First_pokemon', 'Second_pokemon', 'Winner', 'Winner_mark',
       'Type 1_first', 'Type 2_first', 'HP_first', 'Attack_first',
       'Defense_first', 'Sp. Atk_first', 'Sp. Def_first', 'Speed_first',
       'Generation_first', 'Legendary_first', '#_y', 'Type 1_second',
       'Type 2_second', 'HP_second', 'Attack_second', 'Defense_second',
       'Sp. Atk_second', 'Sp. Def_second', 'Speed_second', 'Generation_second',
       'Legendary_second'],
      dtype='object')

In [ ]:
# poke_dummies.columns

In [ ]:
# combats_df[['First_pokemon', 'Second_pokemon', 'Winner']] = combats_df[['First_pokemon', 'Second_pokemon', 'Winner']].astype(str)

In [78]:
temp_df

,#_x,First_pokemon,Second_pokemon,Winner,Winner_mark,Type 1_first,Type 2_first,HP_first,Attack_first,Defense_first,...,Type 1_second,Type 2_second,HP_second,Attack_second,Defense_second,Sp. Atk_second,Sp. Def_second,Speed_second,Generation_second,Legendary_second
0,266,266,298,298,2,15,10,50,64,50,...,9,1,70,70,40,60,40,60,3,0
1,702,702,701,701,2,9,5,91,90,72,...,15,5,91,129,90,72,90,108,5,1
2,191,191,668,668,2,4,7,55,40,85,...,14,12,75,75,75,125,95,40,5,0
3,237,237,683,683,2,6,12,40,40,40,...,2,12,77,120,90,60,90,48,5,0
4,151,151,231,151,1,15,18,70,60,125,...,0,16,20,10,230,10,230,5,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,707,707,126,707,1,2,6,100,120,100,...,17,12,30,40,70,70,25,60,1,0
49996,589,589,664,589,1,10,12,60,85,40,...,3,12,35,55,40,45,40,60,5,0
49997,303,303,368,368,2,17,7,60,50,100,...,12,12,73,115,60,60,60,90,3,0
49998,109,109,89,109,1,3,12,40,30,50,...,3,17,25,35,70,95,55,45,1,0


In [79]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   #_x                50000 non-null  int64
 1   First_pokemon      50000 non-null  int64
 2   Second_pokemon     50000 non-null  int64
 3   Winner             50000 non-null  int64
 4   Winner_mark        50000 non-null  int64
 5   Type 1_first       50000 non-null  int64
 6   Type 2_first       50000 non-null  int64
 7   HP_first           50000 non-null  int64
 8   Attack_first       50000 non-null  int64
 9   Defense_first      50000 non-null  int64
 10  Sp. Atk_first      50000 non-null  int64
 11  Sp. Def_first      50000 non-null  int64
 12  Speed_first        50000 non-null  int64
 13  Generation_first   50000 non-null  int64
 14  Legendary_first    50000 non-null  int64
 15  #_y                50000 non-null  int64
 16  Type 1_second      50000 non-null  int64
 17  Type 2_secon

In [80]:
temp_df.isnull().sum()

#_x                  0
First_pokemon        0
Second_pokemon       0
Winner               0
Winner_mark          0
Type 1_first         0
Type 2_first         0
HP_first             0
Attack_first         0
Defense_first        0
Sp. Atk_first        0
Sp. Def_first        0
Speed_first          0
Generation_first     0
Legendary_first      0
#_y                  0
Type 1_second        0
Type 2_second        0
HP_second            0
Attack_second        0
Defense_second       0
Sp. Atk_second       0
Sp. Def_second       0
Speed_second         0
Generation_second    0
Legendary_second     0
dtype: int64

In [82]:
temp_df.to_csv('./pokemon_preprocessed.csv', index=False)